In [39]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# import plotly.express as px

import statsmodels.api as sm

from IPython.display import display

## Limpeza dos dados

In [40]:
df0 = pd.read_csv('sao-paulo-properties-april-2019.csv')
display(df0.head(3))
display(df0.shape)

,Price,Condo,Size,Rooms,Toilets,Suites,Parking,Elevator,Furnished,Swimming Pool,New,District,Negotiation Type,Property Type,Latitude,Longitude
0,930,220,47,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.543138,-46.479486
1,1000,148,45,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.550239,-46.480718
2,1000,100,48,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.542818,-46.485665


(13640, 16)

In [41]:
rent = df0['Negotiation Type'] == 'rent'
df = df0.loc[rent,['Price', 'Condo', 'Size', 'Rooms', 'Toilets', 'Suites', 'Parking', 'Elevator', 'Furnished', 'Swimming Pool', 'New', 'District']]

# df para regressão linear
df_rl = df.drop(columns='District', axis=1)

In [42]:
df['District'] = df['District'].astype('category')

df['Swimming Pool'] = df['Swimming Pool'].astype('category')
df['New'] = df['New'].astype('category')
df['Furnished'] = df['Furnished'].astype('category')
df['Elevator'] = df['Elevator'].astype('category')

In [43]:
df['Swimming Pool'].cat.categories = ['No', 'Yes']
df['Furnished'].cat.categories = ['No', 'Yes']
df['Elevator'].cat.categories = ['No', 'Yes']
df['New'].cat.categories = ['No', 'Yes']
df['Swimming Pool'].cat.categories = ['No', 'Yes']

In [44]:
df[['District','City']] = df['District'].str.split('/', expand = True)
display(df.City.unique())

array(['São Paulo'], dtype=object)

In [45]:
df = df.drop(columns='City', axis=1)
df['District'] = df['District'].astype('category')

## Análize dos dados

## test train

In [46]:
X_test = pd.read_csv('X_test.csv')
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
y_test = pd.read_csv('y_test.csv')

## Regressão Linear

`Explicaremos aqui todos os nossos passos, quem quiser entender melhor como funciona regressão linear pode ler sobre no seguinte site: https://www.ibm.com/br-pt/analytics/learn/linear-regression`

In [47]:
def regress(Y, X):

    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp)
    r = model.fit()

    return r

In [48]:
X_train['Swimming Pool'] = X_train['Swimming Pool'].astype('category')
X_train['New'] = X_train['New'].astype('category')
X_train['Furnished'] = X_train['Furnished'].astype('category')
X_train['Elevator'] = X_train['Elevator'].astype('category')

X_train['Swimming Pool'].cat.categories = ['0', '1']
X_train['Furnished'].cat.categories = ['0', '1']
X_train['Elevator'].cat.categories = ['0', '1']
X_train['New'].cat.categories = ['0', '1']

X_train['Swimming Pool'] = X_train['Swimming Pool'].astype('int64')
X_train['New'] = X_train['New'].astype('int64')
X_train['Furnished'] = X_train['Furnished'].astype('int64')
X_train['Elevator'] = X_train['Elevator'].astype('int64')

`Adapitamos os nossos dados qualitativos para ficarem na forma binária e podermos fazer regressão linear com todas variáveis relevantes`

In [49]:
Y = y_train['Price']
X = X_train[['Condo', 'Size', 'Rooms', 'Toilets', 'Suites', 'Parking', 'Elevator', 'Furnished', 'Swimming Pool', 'New']]
results = regress(Y,X)
results.summary()
# results.pvalues

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.609
Model:                            OLS   Adj. R-squared:                  0.608
Method:                 Least Squares   F-statistic:                     898.8
Date:                Thu, 02 Jun 2022   Prob (F-statistic):               0.00
Time:                        15:13:11   Log-Likelihood:                -52677.
No. Observations:                5782   AIC:                         1.054e+05
Df Residuals:                    5771   BIC:                         1.054e+05
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const           -15.7238    114.334     -0.138      0.891    -239.861     208.413
Condo             0.9004      0.065     13.926      0.000       0.774       1.027
Size             27.6610      1.009     27.427      0.000      25.684      29.638
Rooms          -746.7056     49.616    -15.050      0.000    -843.972    -649.439
Toilets         210.0955     78.942      2.661      0.008      55.340     364.851
Suites          169.4692     89.518      1.893      0.058      -6.020     344.958
Parking         486.9709     57.660      8.446      0.000     373.936     600.006
Elevator        -93.3106     65.273     -1.430      0.153    -221.270      34.649
Furnished      1095.7383     78.900     13.888      0.000     941.065    1250.411
Swimming Pool   223.3972     62.956      3.548      0.000      99.980     346.814
New            -257.3482    897.166     -0.287      0.774   -2016.131    1501.434
==============================================================================
Omnibus:                     5115.530   Durbin-Watson:                   2.030
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           566733.591
Skew:                           3.743   Prob(JB):                         0.00
Kurtosis:                      50.920   Cond. No.                     3.62e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.62e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [50]:
# X = X.drop(columns='Swimming Pool', axis=1)
# results = regress(Y,X)
# results.pvalues

In [51]:
# X = X.drop(columns='Toilets', axis=1)
# results = regress(Y,X)
# results.pvalues

In [52]:
# X = X.drop(columns='Parking', axis=1)
# results = regress(Y,X)
# results.pvalues

In [53]:
# X = X.drop(columns='Rooms', axis=1)
# results = regress(Y,X)
# results.pvalues

In [54]:
X = X.drop(columns='Furnished', axis=1)
results = regress(Y,X)
results.pvalues

const             4.005885e-02
Condo             1.115989e-49
Size             2.455109e-145
Rooms             1.013415e-63
Toilets           5.175198e-03
Suites            3.256055e-02
Parking           5.390853e-18
Elevator          6.011903e-01
Swimming Pool     1.750132e-08
New               8.033180e-01
dtype: float64

`Calculamos o valor p entre as variáveis e retiramos uma a uma as seguintas variáveis nessa ordem: New, Elevator, Suites, Swimming Pool, Toilets, Parking, Rooms, Furnished.`

In [55]:
%matplotlib notebook


from mpl_toolkits.mplot3d import Axes3D

from scipy import stats 

In [56]:
beta0_hat = results.params[0]  # const
beta1_hat = results.params[1]  # Condo
beta2_hat = results.params[2]  # Size
beta3_hat = results.params[3]  # Size
beta4_hat = results.params[4]  # Size
beta5_hat = results.params[5]  # Size
beta6_hat = results.params[6]  # Size
beta7_hat = results.params[7]  # Size
beta8_hat = results.params[8]  # Size
beta9_hat = results.params[9]  # Size
beta10_hat = results.params[10]  # Size

IndexError: index 10 is out of bounds for axis 0 with size 10

`"Puxamos" do resultado da regressão linear o β0, β do Condo e o β do Size`

In [ ]:
def f(feature1, feature2, feature3, feature4, feature5, feature6, feature7, feature8, feature9, feature10,):
    return beta0_hat + beta1_hat*feature1 + beta2_hat*feature2 + beta3_hat*feature3 + beta4_hat*feature4 + beta5_hat*feature5 + beta6_hat*feature6 + beta7_hat*feature7 + beta8_hat*feature8 + beta9_hat*feature9 + beta10_hat*feature10

In [ ]:
npoints = 100
feature1 = X['Condo']
feature2 = X['Size']
feature3 = X['Rooms']
feature4 = X['Toilets']
feature5 = X['Suites']
feature6 = X['Parking']
feature7 = X['Elevator']
feature8 = X['Furnished']
feature9 = X['Swimming Pool']
feature10 = X['New']
x_vc = np.linspace(feature1.min(), feature1.max(), npoints)
y_vc = np.linspace(feature2.min(), feature2.max(), npoints)

eixo_x, eixo_y = np.meshgrid(x_vc, y_vc)

fx = f(eixo_x, eixo_y)

TypeError: f() missing 8 required positional arguments: 'feature3', 'feature4', 'feature5', 'feature6', 'feature7', 'feature8', 'feature9', and 'feature10'

In [ ]:
plt3d = plt.figure().gca(projection="3d");
plt3d.plot_surface(eixo_x, eixo_y, fit_z);
plt3d.scatter(feature1, feature2, Y, color="r");
plt3d.set_xlabel("feature1");
plt3d.set_ylabel("feature2");
plt3d.set_zlabel("Y");
plt.show();

In [ ]:
X = X_train['Size']
Y = y_train['Price']

med_x = X.mean()
med_y = Y.mean()

beta1 = X.cov(Y)/X.var()
beta0 = med_y-beta1*med_x

x_vec = np.linspace(X.min(), X.max(), 1000)
y_vec = beta0 + beta1*x_vec

fig = plt.figure(figsize=(10, 2.6))

plt.subplot(1,2,1)
plt.scatter(X, Y, alpha=0.4, color='darkmagenta')
plt.plot(x_vec, y_vec)
plt.xlabel('Size', fontsize=13)
plt.ylabel('Price', fontsize=13)

e = X - beta0 - beta1*X

plt.subplot(1,2,2)
stats.probplot(e, dist='norm', plot=plt)
plt.show()

In [ ]:
test = X_train['Size']<=590

X = X_train.loc[test, ['Size']]
Y = y_train.loc[test, ['Price']]

med_x = X.Size.mean()
med_y = Y.Price.mean()

beta1 = X.Size.cov(Y.Price)/X.Size.var()
beta0 = med_y-beta1*med_x
x_vec = np.linspace(X.Size.min(), X.Size.max(), 1000)
y_vec = beta0 + beta1*x_vec

fig = plt.figure(figsize=(10, 2.6))

plt.subplot(121)
plt.scatter(X.Size, Y.Price, alpha=0.4, color='darkmagenta')
plt.plot(x_vec, y_vec)
plt.xlabel('Size', fontsize=13)
plt.ylabel('Price', fontsize=13)

e = X.Size - beta0 - beta1*X.Size

plt.subplot(122)
stats.probplot(e, dist='norm', plot=plt)
plt.show()

In [ ]:
X = X_train['Condo']
Y = y_train['Price']

med_x = X.mean()
med_y = Y.mean()

beta1 = X.cov(Y)/X.var()
beta0 = med_y-beta1*med_x

x_vec = np.linspace(X.min(), X.max(), 1000)
y_vec = beta0 + beta1*x_vec

fig = plt.figure(figsize=(10, 2.6))

plt.subplot(121)
plt.scatter(X, Y, alpha=0.4, color='darkmagenta')
plt.plot(x_vec, y_vec)
plt.xlabel('Condo', fontsize=13)
plt.ylabel('Price', fontsize=13)

e = X - beta0 - beta1*X

plt.subplot(122)
stats.probplot(e, dist='norm', plot=plt)
plt.show()

In [ ]:
test = X_train['Condo']<=7000

X = X_train.loc[test, ['Condo']]
Y = y_train.loc[test, ['Price']]

med_x = X.Condo.mean()
med_y = Y.Price.mean()

beta1 = X.Condo.cov(Y.Price)/X.Condo.var()
beta0 = med_y-beta1*med_x
x_vec = np.linspace(X.Condo.min(), X.Condo.max(), 1000)
y_vec = beta0 + beta1*x_vec

fig = plt.figure(figsize=(10, 2.6))

plt.subplot(121)
plt.scatter(X.Condo, Y.Price, alpha=0.4, color='darkmagenta')
plt.plot(x_vec, y_vec)
plt.xlabel('Condo', fontsize=13)
plt.ylabel('Price', fontsize=13)

e = X.Condo - beta0 - beta1*X.Condo

plt.subplot(122)
stats.probplot(e, dist='norm', plot=plt)
plt.show()

In [ ]:
test = X_train['Condo']<=7000
X = X_train.loc[test, ['Condo']]
Y = y_train.loc[test, ['Price']]

test = X.Condo > 0
X = X.loc[test, ['Condo']]
Y = Y.loc[test, ['Price']]

med_x = X.Condo.mean()
med_y = Y.Price.mean()

beta1 = X.Condo.cov(Y.Price)/X.Condo.var()
beta0 = med_y-beta1*med_x
x_vec = np.linspace(X.Condo.min(), X.Condo.max(), 1000)
y_vec = beta0 + beta1*x_vec

fig = plt.figure(figsize=(10, 2.6))

plt.subplot(121)
plt.scatter(X.Condo, Y.Price, alpha=0.4, color='darkmagenta')
plt.plot(x_vec, y_vec)
plt.xlabel('Condo', fontsize=13)
plt.ylabel('Price', fontsize=13)

e = X.Condo - beta0 - beta1*X.Condo

plt.subplot(122)
stats.probplot(e, dist='norm', plot=plt)
plt.show()

### Referêcias

- Base de dados: https://www.kaggle.com/code/juxwzera/predictions-for-price/data
- Análise de dados e previsão de preços: https://www.kaggle.com/code/juxwzera/predictions-for-price
- Algumas funções do statsmodels.api: https://www.statsmodels.org/stable/gettingstarted.html
- Exemplo de regressão linear: https://pt.linkedin.com/pulse/regressão-linear-simples-e-múltipla-em-python-100-prático-santos